In [1]:
import sys, os
import requests
import selenium
from selenium import webdriver
import requests
from pandas import DataFrame
from bs4 import BeautifulSoup
import re
from datetime import datetime
import pickle, progressbar, json, glob, time
from tqdm import tqdm
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import numpy as np
import csv
import time

In [32]:
def crawling_main_text(url):

    specialChars = "\n\r\t" 
    
    req = requests.get(url)
    req.encoding = None
    soup = BeautifulSoup(req.text, 'html.parser')
    
    # 연합뉴스
    if ('://yna' in url) | ('app.yonhapnews' in url): 
        main_article = soup.find('article', {'class':'story-news article'})
        if main_article == None:
            main_article = soup.find('div', {'class' : 'article-txt'})
            
        text = main_article.text
        
    # MBC 
    elif '//imnews.imbc' in url: 
        text = soup.find('div', {'class' : 'news_txt'}, {'itemprop' : 'articleBody'}).text
        text = text.strip()
        text = text.replace("\r","")
        
#     # 매일경제(미라클), req.encoding = None 설정 필요
#     elif 'mirakle.mk' in url:
#         text = soup.find('div', {'class' : 'view_txt'}).text
        
    # 매일경제, req.encoding = None 설정 필요
#     elif 'mk.co' in url:
#         text = soup.find('div', {'class' : ['art_txt', 'read_txt', 'view_txt']})
#         text.find('script').decompose()
#         text = text.text
#         text = text.strip()
#         for specialChar in specialChars:
#             text = text.replace(specialChar, '')
        
    # SBS
    elif 'news.sbs' in url:
        text = soup.find('div', {'itemprop' : 'articleBody'}).text
    
    # KBS
    elif 'news.kbs' in url:
        text = soup.find('div', {'id' : 'cont_newstext'}).text
        
    # JTBC
    elif 'news.jtbc' in url:
        text = soup.find('div', {'class' : 'article_content'}).text
        
    # 한겨레
    elif 'hani.co' in url:
        text = soup.find('div', {'class' : 'text'}).text
        
    # 동아일보
    elif 'donga.com' in url:
        text = soup.find('div', {'class' : 'article_txt'}).text
        
#     # 조선일보
#     elif 'chosun' in url:
#         main_article = soup.find('section', {'itemprop':'articleBody'})
#         text = main_article.text
        
    # 중앙일보
    elif 'joongang.co' in url:
        text = soup.find('div', {'class' : 'article_body fs3'}).text
        
    #국민일보
    elif 'kmib.co' in url:
        text = soup.find('div', {'class' : 'tx'}).text
        
    else:
        text == None
        
    return text.replace('\n','').replace('\r','').replace('<br>','').replace('\t','')

In [98]:
sleep_sec=0.5

news_num = int(input('수집 뉴스의 수(숫자만 입력) : '))
query = input("검색어 입력: ")
s_date = input("시작 날짜 입력:")
e_date = input("끝나는 날짜 입력:")
sort = "0"
# sort = input("뉴스 검색 방식 입력(관련도순=0  최신순=1  오래된순=2): ")    #관련도순=0  최신순=1  오래된순=2
# media = input("언론사 입력:")

s_from = s_date.replace(".","")
e_to = e_date.replace(".","")
page = 0

options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

news_url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort="+ sort +"&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)

# 웹 페이지 열기
driver.get(news_url)
time.sleep(sleep_sec)

media = 'MBC'

print(f'검색할 언론사 : {media}')

if media == 'KBS':
    office_section_code= "15"
    news_office_checked = "1056"
    next_page_url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sm=tab_opt&sort="+ sort +"&photo=0&field=0&pd=3&ds=" + s_date + "&de=" + e_date + "&docid=&related=0&mynews=1&office_type=2&office_section_code="+ office_section_code +"&news_office_checked="+ news_office_checked +"&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "&is_sug_officeid=0"
elif media == 'MBC':
    office_section_code= "15"
    news_office_checked = "1214"
    next_page_url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sm=tab_opt&sort="+ sort +"&photo=0&field=0&pd=3&ds=" + s_date + "&de=" + e_date + "&docid=&related=0&mynews=1&office_type=2&office_section_code="+ office_section_code +"&news_office_checked="+ news_office_checked +"&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "&is_sug_officeid=0"
elif media == 'SBS':
    office_section_code= "15"
    news_office_checked = "1055"
    next_page_url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sm=tab_opt&sort="+ sort +"&photo=0&field=0&pd=3&ds=" + s_date + "&de=" + e_date + "&docid=&related=0&mynews=1&office_type=2&office_section_code="+ office_section_code +"&news_office_checked="+ news_office_checked +"&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "&is_sug_officeid=0"
elif media == 'JTBC':
    office_section_code= "15"
    news_office_checked = "1437"
    next_page_url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sm=tab_opt&sort="+ sort +"&photo=0&field=0&pd=3&ds=" + s_date + "&de=" + e_date + "&docid=&related=0&mynews=1&office_type=2&office_section_code="+ office_section_code +"&news_office_checked="+ news_office_checked +"&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "&is_sug_officeid=0"
elif media == '매일경제':
    office_section_code= "3"
    news_office_checked = "1009"
    next_page_url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sm=tab_opt&sort="+ sort +"&photo=0&field=0&pd=3&ds=" + s_date + "&de=" + e_date + "&docid=&related=0&mynews=1&office_type=2&office_section_code="+ office_section_code +"&news_office_checked="+ news_office_checked +"&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "&is_sug_officeid=0"
elif media == '한겨레':
    office_section_code= "14"
    news_office_checked = "1028"
    next_page_url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sm=tab_opt&sort="+ sort +"&photo=0&field=0&pd=3&ds=" + s_date + "&de=" + e_date + "&docid=&related=0&mynews=1&office_type=2&office_section_code="+ office_section_code +"&news_office_checked="+ news_office_checked +"&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "&is_sug_officeid=0"
elif media == '동아일보':
    office_section_code= "3"
    news_office_checked = "1020"
    next_page_url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sm=tab_opt&sort="+ sort +"&photo=0&field=0&pd=3&ds=" + s_date + "&de=" + e_date + "&docid=&related=0&mynews=1&office_type=2&office_section_code="+ office_section_code +"&news_office_checked="+ news_office_checked +"&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "&is_sug_officeid=0"
elif media == '조선일보':
    office_section_code= "9"
    news_office_checked = "1023"
    next_page_url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sm=tab_opt&sort="+ sort +"&photo=0&field=0&pd=3&ds=" + s_date + "&de=" + e_date + "&docid=&related=0&mynews=1&office_type=2&office_section_code="+ office_section_code +"&news_office_checked="+ news_office_checked +"&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "&is_sug_officeid=0"
elif media == '중앙일보':
    office_section_code= "9"
    news_office_checked = "1025"
    next_page_url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sm=tab_opt&sort="+ sort +"&photo=0&field=0&pd=3&ds=" + s_date + "&de=" + e_date + "&docid=&related=0&mynews=1&office_type=2&office_section_code="+ office_section_code +"&news_office_checked="+ news_office_checked +"&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "&is_sug_officeid=0"
elif media == '국민일보':
    office_section_code= "1"
    news_office_checked = "1005"
    next_page_url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sm=tab_opt&sort="+ sort +"&photo=0&field=0&pd=3&ds=" + s_date + "&de=" + e_date + "&docid=&related=0&mynews=1&office_type=2&office_section_code="+ office_section_code +"&news_office_checked="+ news_office_checked +"&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "&is_sug_officeid=0"
elif media == '연합뉴스':
    office_section_code= "8"
    news_office_checked = "1001"
    next_page_url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sm=tab_opt&sort="+ sort +"&photo=0&field=0&pd=3&ds=" + s_date + "&de=" + e_date + "&docid=&related=0&mynews=1&office_type=2&office_section_code="+ office_section_code +"&news_office_checked="+ news_office_checked +"&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "&is_sug_officeid=0"
else:
    pbar.close()        
    print('\nurl설정이 되지 않았습니다.\n' + '=' * 100)
    
    
date_list_1 = []
date_list_2 = []
news_dict = {}
contents_dict = {}
title_dict = {}
url_dict = {}
date_dict = {}

idx = 0


req = requests.get(next_page_url.format(query))
soup = BeautifulSoup(req.text, 'html.parser')

print('크롤링 시작')

pbar = tqdm(total=news_num ,leave = True)

while idx < news_num:
    
    table = soup.find('ul',{'class' : 'list_news'})
    li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
    area_list = [li.find('div', {'class' : 'news_area'}) for li in li_list]
    a_list = [area.find('a', {'class' : 'news_tit'}) for area in area_list]
    
    for n in a_list[:min(len(a_list), news_num-idx)]:
        news_dict[idx] = {'title' : n.get('title'),
                          'url' : n.get('href') }
        
      
    urls = [item['url'] for item in news_dict.values()]
    contents = crawling_main_text(urls[idx])
    contents_dict[idx] = {'contents' : contents}
  
    dates_info = [ date_info.get_text() for date_info in soup.find_all('span', attrs={'class':'info'})]
    
    for date_info in dates_info:
        if re.search(r'\d+.\d+.\d+.', date_info) != None:
            date_list_1.append(date_info)
           
    date_list_2 += date_list_1
        
    idx += 1


print('크롤링 완료')

date_list_3 = []
date_list_3 = date_list_2[0:news_num]

query_2 = query.replace('"', '')

print('데이터프레임 변환\n')
news_df = DataFrame(news_dict).T
contents_df = DataFrame(contents_dict).T
date_df = pd.DataFrame(date_list_3, columns = ['date'])
news_df_final = pd.concat([date_df, news_df, contents_df], axis = 1)
csv_file_name = '네이버뉴스_{}_{}개_{}_{}~{}'.format(media, news_num, query_2, s_date, e_date)
news_df_final.to_csv("./{}.csv".format(csv_file_name), encoding='utf-8-sig',index=False)

수집 뉴스의 수(숫자만 입력) : 10
검색어 입력: 금융
시작 날짜 입력:2022.01.01
끝나는 날짜 입력:2022.12.31
검색할 언론사 : MBC
크롤링 시작



  0%|                                                                                           | 0/50 [07:15<?, ?it/s]


크롤링 완료
데이터프레임 변환



In [68]:
url_dict

{0: {'url': 'https://imnews.imbc.com/news/2022/politics/article/6337571_35666.html'}}

In [76]:
news_dict

{0: {'title': '문 대통령 "오미크론이 가장 힘든 고비‥비상한 각오로 총력"',
  'url': 'https://imnews.imbc.com/news/2022/politics/article/6337571_35666.html'},
 1: {'title': '문 대통령, 자가검사키트 공장·선별검사소 방문‥"오미크론 대응에 집중"',
  'url': 'https://imnews.imbc.com/news/2022/politics/article/6337491_35666.html'},
 2: {'title': '문 대통령 "北 중거리 미사일, \'핵·ICBM 유예\' 모라토리움 파기 근처까지"',
  'url': 'https://imnews.imbc.com/news/2022/politics/article/6337450_35666.html'},
 3: {'title': '문 대통령, 北 미사일 관련 NSC 전체 회의 긴급 소집‥1년 만에 주재',
  'url': 'https://imnews.imbc.com/news/2022/politics/article/6337432_35666.html'},
 4: {'title': '李·尹 토론회 열공‥이재명 "국민통합 대통령", 윤석열 "사드 추가배치"',
  'url': 'https://imnews.imbc.com/replay/2022/nwdesk/article/6337514_35744.html'}}

In [95]:
data = {
    0: {'title': '문 대통령 "오미크론이 가장 힘든 고비‥비상한 각오로 총력"',
        'url': 'https://imnews.imbc.com/news/2022/politics/article/6337571_35666.html'},
    1: {'title': '문 대통령, 자가검사키트 공장·선별검사소 방문‥"오미크론 대응에 집중"',
        'url': 'https://imnews.imbc.com/news/2022/politics/article/6337491_35666.html'},
    2: {'title': '문 대통령 "北 중거리 미사일, \'핵·ICBM 유예\' 모라토리움 파기 근처까지"',
        'url': 'https://imnews.imbc.com/news/2022/politics/article/6337450_35666.html'},
    3: {'title': '문 대통령, 北 미사일 관련 NSC 전체 회의 긴급 소집‥1년 만에 주재',
        'url': 'https://imnews.imbc.com/news/2022/politics/article/6337432_35666.html'},
    4: {'title': '李·尹 토론회 열공‥이재명 "국민통합 대통령", 윤석열 "사드 추가배치"',
        'url': 'https://imnews.imbc.com/replay/2022/nwdesk/article/6337514_35744.html'}
}

urls = [item['url'] for item in data.values()]

print(urls)

['https://imnews.imbc.com/news/2022/politics/article/6337571_35666.html', 'https://imnews.imbc.com/news/2022/politics/article/6337491_35666.html', 'https://imnews.imbc.com/news/2022/politics/article/6337450_35666.html', 'https://imnews.imbc.com/news/2022/politics/article/6337432_35666.html', 'https://imnews.imbc.com/replay/2022/nwdesk/article/6337514_35744.html']


In [83]:
data.values()

dict_values([{'title': '문 대통령 "오미크론이 가장 힘든 고비‥비상한 각오로 총력"', 'url': 'https://imnews.imbc.com/news/2022/politics/article/6337571_35666.html'}, {'title': '문 대통령, 자가검사키트 공장·선별검사소 방문‥"오미크론 대응에 집중"', 'url': 'https://imnews.imbc.com/news/2022/politics/article/6337491_35666.html'}, {'title': '문 대통령 "北 중거리 미사일, \'핵·ICBM 유예\' 모라토리움 파기 근처까지"', 'url': 'https://imnews.imbc.com/news/2022/politics/article/6337450_35666.html'}, {'title': '문 대통령, 北 미사일 관련 NSC 전체 회의 긴급 소집‥1년 만에 주재', 'url': 'https://imnews.imbc.com/news/2022/politics/article/6337432_35666.html'}, {'title': '李·尹 토론회 열공‥이재명 "국민통합 대통령", 윤석열 "사드 추가배치"', 'url': 'https://imnews.imbc.com/replay/2022/nwdesk/article/6337514_35744.html'}])